# 분류
- 26일 수업 : 회귀(Regression)
  + 수치 예측
  + 평가지표 : RMSLE (점수가 낮게 나올수록 좋은 모델)
- 27일 수업 : 분류(Classification)
  + 범주 예측 (이진 분류 vs 다중 분류)
    - 이진 분류 : 남여 / 대출연체여부
    - 다중 분류 : 등급 분류 / 이미지와 품종 분류 등

In [1]:
# 붓꽃 분류
from sklearn import datasets
import numpy as np

iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target

# X.shape, y.shape
# 다중 분류
# 0 = setosa, 1 = versicolor, 2 = virginica

print(np.unique(y))

[0 1 2]


- 26일 코드 X_val
  + 모델 한번에 결정 안함(검증 필요, 실험)
  + 테스트 데이터가 확보가 된 상태
- 27일 코드 X_test
  + 테스트 데이터가 없음, 그래서 임의로 만듦
  + 모델 검증 안함, 바로 사용하겠음

In [2]:
# 데이터를 훈련 세트와 테스트 세트로 분할
from sklearn.model_selection import train_test_split

# X: 특성 데이터, y: 레이블 데이터
# test_size=0.3: 테스트 세트 비율 30%
# random_state=1: 재현 가능성을 위한 랜덤 시드
# stratify=y: 계층적 샘플링으로 클래스 비율 유지(층화 추출)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 1, stratify = y
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((105, 2), (45, 2), (105,), (45,))

In [3]:
# 전체 데이터셋의 클래스별 샘플 개수 출력
print('y의 레이블 카운트:', np.bincount(y))
# 훈련 데이터셋의 클래스별 샘플 개수 출력
print('y_train의 레이블 카운트:', np.bincount(y_train))
# 테스트 데이터셋의 클래스별 샘플 개수 출력
print('y_test의 레이블 카운트:', np.bincount(y_test))

y의 레이블 카운트: [50 50 50]
y_train의 레이블 카운트: [35 35 35]
y_test의 레이블 카운트: [15 15 15]


- 특성 표준화

In [4]:
# 모든 수치 데이터를 표준화 평균 0, 표준편차 1로 만들겠음
# 단위가 서로 다를 때 한다.
# linear_model 활용 시 필수
# 결정트리와 같은 비선형 모델은 특성표준화 할 필요가 없음
from sklearn.preprocessing import StandardScaler

sc = StandardScaler() # 인스턴스 생성
sc.fit(X_train)

# 훈련 데이터와 테스트 데이터를 표준화
# 테스트 데이터는 미지의 데이터 (전체 모수를 모르는 데이터)
# 연구자가 아는 건 훈련 데이터만 파악된 상태
X_train_std = sc.transform(X_train)

X_test_std = sc.transform(X_test)

In [5]:
# sc = StandardScaler() # 인스턴스 생성
# sc.fit(X_train)
# X_train_std = sc.transform(X_train)

# sc.fit(X_test) # 나는 미래의 데이터도 이미 다 알고 있음
# X_test_std = sc.transform(X_test)

# # 위 코드로 진행 시, 과대적합 문제 발생
# # Data Leakage 용어

In [6]:
# 모델 만들기
from sklearn.linear_model import Perceptron

# eta0 = 0.1 학습률 설정
# 하이퍼파라미터 튜닝
ppn = Perceptron(eta0 = 0.1, random_state = 1)
ppn.fit(X_train_std, y_train)

Perceptron(eta0=0.1, random_state=1)

In [7]:
# 테스트 데이터에 대한 예측 수행
y_pred = ppn.predict(X_test_std)
y_pred

# 실제 레이블과 예측값이 다른 샘플의 개수 출력
# 1개가 다름
(y_test != y_pred).sum()

np.int64(1)

In [8]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9777777777777777

In [9]:
import seaborn as sns

# 분류모델 만들기
tips = sns.load_dataset("tips")
tips.head()

# size 범주를 예측하는 모델 만들기
# 가이드
# 문자열 데이터가 많음 ==> 인코딩 변환
# 수치 데이터는 ==> 표준화
# 훈련 데이터 / 테스트 데이터 분리
tips['size'].unique() # 다중분류
tips['size'].value_counts() # 다중분류

size
2    156
3     38
4     37
5      5
1      4
6      4
Name: count, dtype: int64

In [10]:
import pandas as pd
import seaborn as sns

# 데이터 불러오기
tips = sns.load_dataset("tips")

# 파생 변수 생성
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips['bill_per_person'] = tips['total_bill'] / tips['size']

# 확인
print(tips.head())

   total_bill   tip     sex smoker  day    time  size   tip_pct  \
0       16.99  1.01  Female     No  Sun  Dinner     2  0.059447   
1       10.34  1.66    Male     No  Sun  Dinner     3  0.160542   
2       21.01  3.50    Male     No  Sun  Dinner     3  0.166587   
3       23.68  3.31    Male     No  Sun  Dinner     2  0.139780   
4       24.59  3.61  Female     No  Sun  Dinner     4  0.146808   

   bill_per_person  
0         8.495000  
1         3.446667  
2         7.003333  
3        11.840000  
4         6.147500  


In [15]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron

tips = sns.load_dataset("tips")

categorical_features = ['sex', 'smoker', 'day', 'time']
numeric_features = ['total_bill', 'tip']

X = tips[numeric_features + categorical_features]
y = tips['size']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 1. 수치형 데이터 전처리
scaler = StandardScaler()
X_train_numeric = scaler.fit_transform(X_train[numeric_features])
X_test_numeric = scaler.transform(X_test[numeric_features])

# 2. 범주형 데이터 one-hot 인코딩 (학습 컬럼 기억)
X_train_categoricals = pd.get_dummies(X_train[categorical_features])
X_test_categoricals = pd.get_dummies(X_test[categorical_features])

# ===> 컬럼 순서 저장 <===
cat_columns = X_train_categoricals.columns

# X_test의 컬럼을 X_train의 컬럼 순서에 맞춰 0으로 채움
X_test_categoricals = X_test_categoricals.reindex(columns=cat_columns, fill_value=0)

# 피처 통합
X_train_processed = np.hstack([X_train_numeric, X_train_categoricals.values])
X_test_processed = np.hstack([X_test_numeric, X_test_categoricals.values])

ppn = Perceptron(eta0=0.1, random_state=1)
ppn.fit(X_train_processed, y_train)

# 예측
y_pred = ppn.predict(X_test_processed)
print((y_test != y_pred).sum())
print(accuracy_score(y_test, y_pred))

# =====================
# 새로운 데이터 예측 (여기서도 반드시 같은 전처리)
# =====================

new_data = pd.DataFrame({
    'total_bill': [20.0],
    'tip': [4.0],
    'sex': ['Male'],
    'smoker': ['No'],
    'day': ['Sun'],
    'time': ['Dinner']
})

# 수치형 변환
new_numeric = scaler.transform(new_data[numeric_features])

# 카테고리형 인코딩
new_categorical = pd.get_dummies(new_data[categorical_features])

# ==> 반드시 학습 컬럼 순서와 맞추기 <==
new_categorical = new_categorical.reindex(columns=cat_columns, fill_value=0)

new_processed = np.hstack([new_numeric, new_categorical.values])

# 예측
print("예측값:", ppn.predict(new_processed))


16
0.673469387755102
예측값: [2]
